**<h1 style="text-align: center;">Quick Intro to LangChain</h1><hr>**

## Setup OpenAI API key

In [2]:
import sys

sys.path.insert(0, 'D:/personal-git')
from key_api import apiKey

import os
os.environ['OPENAI_API_KEY'] = apiKey

## Install necessary modules

In [3]:
!python -m pip install langchain
!python -m pip install OpenAI

## Basic LLM demo

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.6) #Temperature - The scale of creativeness a model can have (range 0 to 1)
answer = llm("Explain what GPT is like I am a undergrad average computer science student")
print(answer)



GPT (Generative Pre-trained Transformer) is a type of language model developed by OpenAI that uses deep learning to produce human-like text. It is a type of artificial intelligence (AI) that can generate natural language from a prompt. GPT can be used for a variety of tasks, including summarizing text, generating text from incomplete input, and answering questions. GPT models are trained on a large corpus of text, such as books, news articles, and Wikipedia pages, and are able to generate text that is indistinguishable from human-written text. GPT models are used in natural language processing (NLP) applications such as chatbots, question answering, and automatic translation.


## Prompt Template

In [5]:
from langchain.prompts import PromptTemplate

squareRootPrompt = PromptTemplate(
    input_variables = ['number'],
    template = "What is the square root of {number}"
)

generatedSqRootPrompt = squareRootPrompt.format(number = input("Enter number: "))
print(generatedSqRootPrompt)
print(llm(generatedSqRootPrompt))

Enter number: 144
What is the square root of 144


The square root of 144 is 12.


## LLM Chain

In [6]:
from langchain.chains import LLMChain

sqRootChain = LLMChain(llm=llm, prompt=squareRootPrompt)
sqRootChain.run(input("Enter a number: "))

Enter a number: 144


'\n\nThe square root of 144 is 12.'

### Simple Sequential Chain

In [19]:
from langchain.chains import SimpleSequentialChain

hotelNamePrompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "A fancy name for a {cuisine} food restaurant"
)

hotelNameChain = LLMChain(llm=llm, prompt=hotelNamePrompt)

hotelMenuPrompt = PromptTemplate(
    input_variables=['name'], 
    template="Create a menu for {name}"
)

hotelMenuChain = LLMChain(llm=llm, prompt=hotelMenuPrompt)

#Here the sequence of the chains matter
hotelChain = SimpleSequentialChain(chains=[hotelNameChain, hotelMenuChain])
answer = hotelChain.run(input("Enter cuisine for the restaurant: "))
print(answer)

Enter cuisine for the restaurant: Indian


Appetizers
- Vegetable Samosas 
- Onion Bhajis
- Paneer Tikka

Main Course
- Chicken Tikka Masala 
- Vegetable Korma
- Lamb Rogan Josh
- Paneer Makhani

Desserts
- Gulab Jamun
- Rasmalai
- Kheer


### Sequential Chain

In [17]:
from langchain.chains import SequentialChain

llm = OpenAI(temperature=0.8)

hotelNamePrompt = PromptTemplate(
    input_variables=['cuisine'],
    template="Suggest a fancy name for a {cuisine} restaurant");

hotelNameChain = LLMChain(llm=llm, prompt=hotelNamePrompt, output_key="hotelName")

menuItemsPrompt = PromptTemplate(
                                input_variables=['hotelName'],
                                template="Suggest some menus for {hotelName}");
menuChain = LLMChain(llm=llm, prompt=menuItemsPrompt, output_key="menuItems")

chain = SequentialChain(
    chains = [hotelNameChain, menuChain],
    input_variables = ['cuisine'],
    output_variables = ['hotelName', 'menuItems']
)

chain({'cuisine':'Mexican'}) 

{'cuisine': 'Mexican',
 'hotelName': '\n\nTaco Palacio Azul',
 'menuItems': '\n\n1. Quesadilla Combo - A flour tortilla filled with cheese, served with Mexican rice and refried beans. \n\n2. Mexican Salad - Mixed greens, grilled peppers, onions, and tomatoes, topped with shredded cheese, avocado, and sour cream. \n\n3. Fajita Platter - Grilled chicken or steak, onions, bell peppers, served with Mexican rice, refried beans and flour tortillas. \n\n4. Chilaquiles - Crispy corn tortillas topped with your choice of chorizo sausage, chicken, or steak, served with refried beans, Mexican rice, and guacamole. \n\n5. Carne Asada - Tender flank steak, cooked to perfection and served with Mexican rice, refried beans, and flour tortillas. \n\n6. Chicken Tortilla Soup - Chicken broth simmered with vegetables and topped with crispy strips of tortilla chips. \n\n7. Queso Fundido - melted cheese, chorizo, and peppers served with tortilla chips. \n\n8. Fish Tacos - Crispy battered cod served in soft fl

## Agents

In [15]:
!python -m pip install wikipedia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11706 sha256=c3badafd48cfa3c9d5a0c9f6a1b8757e96ee78bf557613909d295562eb45f17f
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [16]:
from langchain.agents import AgentType, load_tools, initialize_agent
from langchain.llms import OpenAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True #Displays set by set internal execution
)

agent.run("What is langchain?")



> Entering new AgentExecutor chain...
 I need to investigate what langchain is.
Action: Wikipedia
Action Input: langchain
Observation: Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Sentence embedding
Summary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vectors of real numbers which encodes meaningful semantic information.State of the art embeddings are based on the learned hidden layer representation of dedicated sentence transformer models. BERT pioneered an approach involving the use of a dedicated [CLS] token preprended to the beginning of each sentence inputted into the model; the final hidden state vector of this t

'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). Sentence embedding is a numeric representation of a sentence in the form of a vectors of real numbers, which encodes meaningful semantic information. It is often fine-tuned for use in sentence classification tasks. Other approaches to sentence embedding include aggregating word embeddings, such as Word2vec, into sentence embeddings through the use of approaches such as continuous bag-of-words (CBOW) or vector of locally aggregated word embeddings (VLAWE).'

## Memory

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=hotelNamePrompt, memory=memory)
print(chain.run("Indian"))



Curry Palatial.


In [24]:
print(chain.run("American"))



The Red, White and Blue Bistro


In [26]:
print(chain.memory.buffer)

Human: Indian
AI: 

Curry Palatial.
Human: American
AI: 

The Red, White and Blue Bistro


In [27]:
from langchain.chains import ConversationChain
convo = ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [35]:
from langchain.memory import ConversationBufferWindowMemory

chain = ConversationChain(llm=OpenAI(temperature=0.5), memory=ConversationBufferWindowMemory(k=1))

print(chain.run("What is captial of India?"))
print(chain.run("1+2"))
print(chain.memory.buffer)
print(chain.run("Japan?"))
print(chain.memory.buffer)

 The capital of India is New Delhi. It is located in the northern part of the country and is the seat of the Indian government.
 The answer is 3.
[HumanMessage(content='What is captial of India?', additional_kwargs={}), AIMessage(content=' The capital of India is New Delhi. It is located in the northern part of the country and is the seat of the Indian government.', additional_kwargs={}), HumanMessage(content='1+2', additional_kwargs={}), AIMessage(content=' The answer is 3.', additional_kwargs={})]
 Japan is an island country located in East Asia. It is the world's fourth-largest island country with an area of 377,972 square kilometers. It has a population of over 126 million people, making it the world's tenth-most populous country.
[HumanMessage(content='What is captial of India?', additional_kwargs={}), AIMessage(content=' The capital of India is New Delhi. It is located in the northern part of the country and is the seat of the Indian government.', additional_kwargs={}), HumanMess